In [133]:
import numpy as np
import pandas as pd
from scipy import stats

%precision 3
np.random.seed(1111)

# 統計的仮説検定

In [134]:
# 標本 Data: 14日間のフライドポテトの重さを準備
df = pd.read_csv('../data/ch11_potato.csv')
sample = np.array(df['重さ'])
display(sample)

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [135]:
# 標本平均を求める
s_mean = np.mean(sample)
s_mean

128.451

## 統計的仮説検定とは（statistical hypothesis testing）

### 統計的仮説検定の基本
- フライドポテトの母平均が130gより少ないかどうかを確かめたい。
- 前提として
  - 母集団が正規分布に従っている
  - 母分散は９
とわかっている。

母平均が「130g」という仮定をすると、標本平均は、N(130, 9)に従うことになる

In [136]:
# 標本平均が P(標本平均<=x)=0.05 を満たす x を考える
rv = stats.norm(130, np.sqrt(9 / 14))
rv.isf(0.95)

128.681

標本平均が 126.681g 以下の重さになることは５％の確率でしか発生しない = 標本平均が 128.451g となったのは５％の確率でしか発生しない珍事な出来事といえる。
よって「母平均が 130g より少ない」と考え結論づける。

In [137]:
# 帰無仮説「母平均 130g」
# 検定統計量に標本平均を標準化した Z を使用する。
# 検定統計量が臨界値より小さければ帰無仮説を棄却し、そうでないときに帰無仮説を採択する。

z = (s_mean - 130) / np.sqrt(9/14)
print(f'統計検定量: {z:.3f}')

統計検定量: -1.932


In [138]:
rv = stats.norm()

print(f'臨界値: {rv.isf(0.95):.3f}')

臨界値: -1.645


検定統計量と臨界値を比較すると検定統計量のほうが小さい値となっている。
= 帰無仮説は棄却
= 平均は、130g より少ないという結論になる

In [139]:
# ｐ値を使った仮説検定について確認する。

# 検定統計量（標本平均を標準化した Z）からｐ値を求める
print(f'ｐ値: {rv.cdf(z):.3f}')

ｐ値: 0.027


ｐ値が有意水準 0.05 より小さい値になった。
= 帰無仮説は棄却

### 片側検定と両側検定
フライドポテトの「母平均は 130g ではない」という対立仮説で仮説検定を行なう。（※130g より少ない、多いの両側検定）

In [140]:
# 検定統計量を求める
z = (s_mean - 130) / np.sqrt(9/14)
print(f'検定統計量: {z:.3f}（※標本平均を標準化した値）')

検定統計量: -1.932（※標本平均を標準化した値）


In [141]:
# 臨界値を求める
# 両側検定のため標準正規分布の95%区間によって求める
rv = stats.norm()
rv.interval(0.95)

(-1.960, 1.960)

臨界値と統計検定量を比較してみると、統計検定量が採択域にはいっているため帰無仮説は棄却されない。
※両側検定に比べると片側検定は棄却しやすい。

In [142]:
# ｐ値を求める
# 両側検定のｐ値は、上側と下側の両方の面積を考慮し、累積密度関数の値を２倍にすることによって求められる。
print(f'ｐ値（両側検定: {rv.cdf(z) * 2:.3f}')

ｐ値（両側検定: 0.053


ｐ値が有意水準 0.05 より大きくなっていることからも帰無仮説は棄却されない。

### 仮説検定における２つの過誤

#### 第一種の過誤
帰無仮説が正しいときに、帰無仮説を棄却してしまう過誤がどのくらいの割合で発生してしまうのか Simulation してみる。

In [143]:
# 母集団の確率分布は N(130, 9)
rv = stats.norm(130, 3)

In [144]:
# この母集団から 14個の標本を抽出して仮説検定を行なう、という作業を10000回行なう
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9 / 14)
    if z < c:
        cnt += 1

print(f'第一種の過誤を犯す割合（「平均130g」であるにも関わらず「平均は 130g より少ない」と結論づけてしまう割合）: {cnt / n_samples:.3f}')

第一種の過誤を犯す割合（「平均130g」であるにも関わらず「平均は 130g より少ない」と結論づけてしまう割合）: 0.053


#### 第二種の過誤
対立仮説が正しいときに、帰無仮説を採択してしまう過誤（「母平均が 130g より少ない」のに「母平均が 130g より少ない」という結論が得ることができない）がどのくらいの割合で発生するか simulation してみる。

In [145]:
# 母集団のへの確率分布は N(128, 9)
rv = stats.norm(128, 3)

In [146]:
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9 / 14)
    if z >= c:
        cnt += 1

print(f'第二種の過誤を犯してしまう割合; {cnt / n_samples:.3f}')

第二種の過誤を犯してしまう割合; 0.197


## 基本的な仮説検定

### 正規分布の母平均の検定（母分散既知）

In [147]:
# 正規分布の母平均の仮説検定（母分散既知）を関数として実装

def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1 - alpha)

    z = (s_mean - mean0) / np.sqrt(p_var / n)
    if interval[0] <= z <= interval[1]:
        print('帰無仮説を採択')
    else:
        print('帰無仮説を棄却')
    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1 - rv.cdf(z)) * 2
    print(f'ｐ値は{p:.3f}')

In [151]:
# フライドポテトの標本Data で実行
pmean_test(sample, 130, 9)

帰無仮説を採択
ｐ値は0.053


### 正規分布の母分散の検定

In [152]:
# 正規分布の母分散の検定を関数として実装
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n - 1)
    interval = rv.interval(1 - alpha)

    y = (n - 1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('帰無仮説を採択')
    else:
        print('帰無仮説を棄却')
    if y < rv.isf(0.5):
        p = rv.cdf(y) * 2
    else:
        p = (1 - rv.cdf(y)) * 2
    print(f'ｐ値は{p:.3f}')

In [153]:
# フライドポテトの標本Data で実行
pvar_test(sample, 9)

帰無仮説を採択
ｐ値は0.085
